# This notebook calculates the $5\sigma$ magnitude limit of LSST. The model can be fround in https://arxiv.org/abs/0805.2366

In [2]:
import numpy as np
import yaml


In [17]:
settings = {
            "bandNames": {  # provided so you can alias the names of the bands
                "u": "u",
                "g": "g",
                "r": "r",
                "i": "i",
                "z": "z",
                "y": "y",
            },
            "tvis": 30.0,  # exposure time for a single visit in seconds, p12
            "nYrObs": 10.,  # number of years of observations
            "nVisYr": {  # mean number of visits per year in each filter (T1)
                "u": 5.6,
                "g": 8.0,
                "r": 18.4,
                "i": 18.4,
                "z": 16.0,
                "y": 16.0,
            },
            "gamma": {  # band dependent parameter (T2)
                "u": 0.038,
                "g": 0.039,
                "r": 0.039,
                "i": 0.039,
                "z": 0.039,
                "y": 0.039,
            },
            "airmass": 1.2,  # fiducial airmass (T2)
            "extendedSource": 0.0,  # constant added to m5 for extended sources
            "sigmaSys": 0.005,  # expected irreducible error, p26
            "magLim": 30.0,  # dimmest allowed magnitude; dimmer mags set to ndFlag
            "ndFlag": np.nan,  # flag for non-detections (all mags > magLim)
            "m5": {},  # explicit list of m5 limiting magnitudes
            "Cm": {  # band dependent parameter (T2)
                "u": 23.09,
                "g": 24.42,
                "r": 24.44,
                "i": 24.32,
                "z": 24.16,
                "y": 23.73,
            },
            "msky": {  # median zenith sky brightness at Cerro Pachon (T2)
                "u": 22.99,
                "g": 22.26,
                "r": 21.20,
                "i": 20.48,
                "z": 19.60,
                "y": 18.61,
            },
            "theta": {  # median zenith seeing FWHM, arcseconds (T2)
                "u": 0.81,
                "g": 0.77,
                "r": 0.73,
                "i": 0.71,
                "z": 0.69,
                "y": 0.68,
            },
            "km": {  # atmospheric extinction (T2)
                "u": 0.491,
                "g": 0.213,
                "r": 0.126,
                "i": 0.096,
                "z": 0.069,
                "y": 0.170,
            },
            "highSNR": False,
        }

In [27]:
bands = 'ugrizy'

m5s = {band: settings["Cm"][band]
            + 0.50 * (settings["msky"][band] - 21)
            + 2.5 * np.log10(0.7 / settings["theta"][band])
            + 1.25 * np.log10(settings["tvis"] / 30)
            - settings["km"][band] * (settings["airmass"] - 1)
            - settings["extendedSource"] #+ 1.75
            for band in bands
        }

In [55]:
coadded = True  # assuming 10 year accumulative observation
Nsigma = 5  # 5-sigma magnitude

# and the values for gamma
m5 = np.array([m5s[band] for band in 'ugrizy'])

gamma = np.array([settings["gamma"][band] for band in bands])

# get the number of exposures
if coadded:
    nVisYr = np.array([settings["nVisYr"][band] for band in bands])
    nStackedObs = nVisYr * settings["nYrObs"]
else:
    nStackedObs = 1

# get the irreducible system error
if settings["highSNR"]:
    nsrSys = self.settings["sigmaSys"]
else:
    nsrSys = 10 ** (settings["sigmaSys"] / 2.5) - 1

# calculate the square of the random NSR that a single exposure must have
nsrRandSqSingleExp = (1 / Nsigma ** 2 - nsrSys ** 2) * nStackedObs

# calculate the value of x that corresponds to this NSR
# note this is just the quadratic equation,
# applied to NSR^2 = (0.04 - gamma) * x + gamma * x^2
x = (
        (gamma - 0.04)
        + np.sqrt((gamma - 0.04) ** 2 + 4 * gamma * nsrRandSqSingleExp)
     ) / (2 * gamma)

# convert x to a limiting magnitude
limiting_mags = m5 + 2.5 * np.log10(x)

maglim = {bandName: mag for bandName, mag in zip('ugrizy', limiting_mags)}

In [56]:
from yaml import Loader
config_file = '../scripts/config_yamls/LSST_config.yaml'
with open(config_file) as file:
    documents = yaml.load(file, Loader=Loader)
documents['paths']['DATADIR'] = '${HOME}/DATA/mocks/cosmoDC2/LSST_10yr_mock'
documents['photometric_setup']['MAGlims'] = limiting_mags.tolist()

In [57]:
with open('../scripts/config_yamls/LSST_10yr_config.yaml', 'w') as outfile:
    yaml.dump(documents, outfile, default_flow_style=False)